# 1. Download Artigos SciELO

Neste notebook é possível fazer download dos artigos da RBEF que podem ser encontrados no site de [SciELO](https://www.scielo.br/j/rbef/). 

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from tqdm.notebook import tqdm

from utils.scielo import volumes_por_ano, html_do_artigo, artigos_por_volume

## Código

Nesta parte do notebook consiste no código de download dos artigos que estão no site da SciELO e que são da língua portugesa.

In [ ]:
volumes = volumes_por_ano() 
del volumes[2023]  # remove 2023 dos volumes.

In [ ]:
try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos_SciELO')) # cria a pasta dos artigos brutos
except FileExistsError:
    print('A pasta `artigos_brutos_SciELO` já existe!')

In [ ]:
path = os.path.join(os.getcwd(), 'artigos_brutos_SciELO')

In [ ]:
for ano, urls in tqdm(volumes.items()):
    
    dir_path = os.path.join(path, str(ano))
    
    try:
        os.mkdir(dir_path)
    except FileExistsError:
        pass
    
    n = 1
    for volume_url in urls:
        todos = artigos_por_volume(volume_url)
        
        for artigo_url in todos:
            text = html_do_artigo(artigo_url)
            titulo = f'artigo_{ano}_{str(n).rjust(3, "0")}.html'
            path_save = os.path.join(dir_path, titulo)
            
            with open(path_save, 'w', encoding='utf8') as file:
                file.write(text)
            
            n += 1


Depois de fazer o download dos arquivos e salvá-los em uma pasta, podemos criar um Dataset para facilitar a navegação pelo conteúdo.

Após a criação do dataset esses arquivos brutos são apagados.